In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sklearn
from skopt import *
import ray
from ray import tune
from sklearn import *

### Wine DataSet

In [ ]:
X, Y = datasets.load_wine(return_X_y=True)
print(X.shape, Y.shape)
X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(X,Y, test_size=0.2)
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

In [ ]:
def my_func(config, reporter):  # add the reporter parameter
      # report the configuration
    clf = sklearn.neural_network.MLPClassifier(solver='lbfgs', alpha=config["alpha"],
                                            hidden_layer_sizes=config["hidden_layer_sizes"],
                                            max_iter=90000,activation='relu',random_state=1)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    reporter(config, mean_accuracy = sklearn.metrics.accuracy_score(y_test, y_pred))
    print(sklearn.metrics.accuracy_score(y_test, y_pred))
    
tune.register_trainable("my_func", my_func)
ray.shutdown()
ray.init()

res = tune.run_experiments({
    "my_experiment": {
        "run": "my_func",
        "stop": { "mean_accuracy": 0.6 },
        "config": {
            "alpha": tune.grid_search([0.1, 0.2]),
            "hidden_layer_sizes": tune.grid_search([(5,2), (10,2)]),
        }
    }
})